In [ ]:
import pandas as pd
import numpy as np # For the example data generation
from docx import Document
from docx.shared import Pt, Inches # For Pt and Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn # For XML namespace stuff (borders)
from docx.oxml import OxmlElement # For manipulating Oxml directly for borders


# Protocol Deviations

## Functions

In [ ]:
def parse_deviation_category(dvcat_string):
    """
    Extracts the parent category and subcategory description from a DVCAT string.
    The parent category is the text before the first colon (':').
    The subcategory description is the text after the first colon.
    If no colon is present, a default parent category is assigned,
    and the entire string is considered the subcategory description.

    Args:
        dvcat_string (str): The string from the DVCAT column.

    Returns:
        tuple: A tuple containing (parent_category, subcategory_description).
    """
    # we ensure that the input is treated as a string
    dvcat_string = str(dvcat_string)
    # we split the string at the colon at most once and store it in 'parts'
    parts = dvcat_string.split(':', 1)
    # this checks if the len of the parts is more than one, meaning the 
    # split was successful
    if len(parts) > 1:
        # we separate the parent and description into separate variables
        parent = parts[0].strip()
        description = parts[1].strip()
        # we return the first part (parent part) stripped of surrounding whitspace
        return parent, description
    # if no split occured
    # But, this should not occur
    else:
        parent = "Other Deviations"
        description = dvcat_string.strip()
        return dvcat_string


## Load the data from the Excel sheet
Clean data - removing NA cells as the DV dataset contains screen failures that are not actually deviations.
Currently the total number of participants is defined in the code. But ideally we would use some part of the raw data set.

In [ ]:
# load the excel as a pd dataframe
try:
    dv_df = pd.read_excel("Raw_Data/DV.xlsx")
except FileNotFoundError:
    print("Data not found")

# Explicitly convert the 'DVCAT' column to string type.
# .astype(str) will convert numbers, NaNs, etc., to their string representations.
# For example, NaN becomes the string 'nan'.
dv_df['DVCAT_str_temp'] = dv_df['DVCAT'].astype(str)

# replace entries that are empty or contain only whitespace with pd.NA (Pandas' missing value indicator)
# ensure the column is a string and replace None/NA with an empty string
# the regex looks for empty string or strings with whitespace
dv_df['DVCAT_cleaned'] = dv_df['DVCAT_str_temp'].replace(r'^\s*$', np.nan, regex=True).replace('nan', np.nan, regex=False)
rows_before_dropping = len(dv_df)
# Drop rows where the cleaned DVCAT column is now NaN
dv_df.dropna(subset=['DVCAT_cleaned'], inplace=True)
rows_after_dropping = len(dv_df)

if rows_before_dropping > rows_after_dropping:
    print(f"{rows_before_dropping - rows_after_dropping} row(s) with empty, whitespace, or NaN 'DVCAT' were removed.")
else:
    print(f"No rows with empty, whitespace, or NaN 'DVCAT' found to remove (based on DVCAT_cleaned).")

# Define the total number of participants here
total_study_participants = 27
total_deviations_overall = len(dv_df)

## Separate parent and child categories, counts for each

In [ ]:
# Create the parent and subcategory separations

# the .apply() method passes each value from DVCAT to the 
# parse deviation category function
parsed_categories = dv_df['DVCAT'].apply(parse_deviation_category)

# create new columns from the parsed data
dv_df['Parent_Category'] = [item[0] for item in parsed_categories]
dv_df['Subcategory_Description'] = [item[1] for item in parsed_categories]

In [ ]:

deviation_summary_list = []
if 'Parent_Category' in dv_df.columns and 'Subcategory_Description' in dv_df.columns:
    parent_category_counts = dv_df['Parent_Category'].value_counts()

    for parent_cat in parent_category_counts.index:
        parent_df_subset = dv_df[dv_df['Parent_Category'] == parent_cat]
        deviations_in_parent_cat = len(parent_df_subset)
        unique_subjects_in_parent_cat = parent_df_subset['SUBJID'].nunique()
        percentage_subjects_parent = (unique_subjects_in_parent_cat / total_study_participants) * 100
        
        # Percentage of total deviations this parent category represents
        percentage_total_deviations_parent = (deviations_in_parent_cat / total_deviations_overall) * 100

        formatted_n_column_parent = f"{unique_subjects_in_parent_cat} ({percentage_subjects_parent:.1f}%) {deviations_in_parent_cat}"
        
        deviation_summary_list.append({
            'Deviation Category/Subcategory': parent_cat,
            'Number of Deviations (n)': formatted_n_column_parent, # New formatted string
        })

        subcategory_counts_for_this_parent = parent_df_subset['Subcategory_Description'].value_counts()

        for sub_cat_desc, count_devs_in_sub_cat in subcategory_counts_for_this_parent.items():
            # To get unique subjects for this specific subcategory, filter the parent_df_subset further
            sub_cat_df_subset = parent_df_subset[parent_df_subset['Subcategory_Description'] == sub_cat_desc]
            unique_subjects_in_sub_cat = sub_cat_df_subset['SUBJID'].nunique()
            percentage_subjects_sub = (unique_subjects_in_sub_cat / total_study_participants) * 100
            
            # Percentage of total deviations this subcategory represents
            percentage_total_deviations_sub = (count_devs_in_sub_cat / total_deviations_overall) * 100
            
            formatted_n_column_sub = f"{unique_subjects_in_sub_cat} ({percentage_subjects_sub:.1f}%) {count_devs_in_sub_cat}"

            deviation_summary_list.append({
                'Deviation Category/Subcategory': f"    {sub_cat_desc}",
                'Number of Deviations (n)': formatted_n_column_sub, # New formatted string
            })

In [ ]:
if deviation_summary_list:
    summary_df_hierarchical = pd.DataFrame(deviation_summary_list)
    # Rename the columns for clarity if desired, matching the new content
    summary_df_hierarchical.rename(columns={
        'Number of Deviations (n)': 'Number of Participants (% of Total Participants) Number of Occurences',
    }, inplace=True)

    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', 120) # Adjust width for better display
    print("\nTable: Summary of Protocol Deviations by Category")
    print(summary_df_hierarchical.to_string(index=False))
else:
    print("Could not generate the deviation summary table (summary list is empty).")


## Generate Word Document Table

In [ ]:
# --- Function to set specific borders (more robust) ---
def set_cell_borders(cell, **kwargs):
    """
    Set cell's border
    Usage:
    set_cell_borders(cell,
        top={"sz": 6, "val": "single", "color": "000000"},
        bottom={"sz": 6, "val": "single", "color": "000000"},
        left={"val": "nil"}, # To remove a border
        right={"val": "nil"}
    )
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement('w:tcBorders')
        tcPr.append(tcBorders)

    for edge in ('top', 'left', 'bottom', 'right', 'insideH', 'insideV'):
        edge_data = kwargs.get(edge)
        if edge_data:
            tag = f'w:{edge}'
            border = tcBorders.find(qn(tag))
            if border is None:
                border = OxmlElement(tag)
                tcBorders.append(border)
            
            if edge_data.get("val") == "nil": # Special case to remove border
                border.set(qn('w:val'), "nil")
                for att in ["sz", "color", "space", "shadow"]: # remove other attributes if setting to nil
                    if qn(f'w:{att}') in border.attrib:
                        del border.attrib[qn(f'w:{att}')]

            else:
                for k, v in edge_data.items():
                    border.set(qn(f'w:{k}'), str(v))


# --- STEP 5: Create Word Document with the Table (Advanced Styling) ---

if 'summary_df_hierarchical' in locals() and not summary_df_hierarchical.empty:
    try:
        document = Document()
        document.add_paragraph('Protocol Deviation Summary')

        font_name = 'Calibri' 
        font_size = Pt(10)
        parent_indent = Inches(0)    # No indent for parent
        sub_cat_indent = Inches(0.2) # Indent for subcategories (adjust as needed)

        table = document.add_table(rows=summary_df_hierarchical.shape[0] + 1, cols=summary_df_hierarchical.shape[1])
        # Don't apply 'Table Grid' style if we want full control over borders.
        # Or, apply it and then try to override. Let's try setting borders manually.
        
        # Set all borders to nil first for the whole table
        # This is more involved; python-docx doesn't have a simple table.set_all_borders_none()
        # We will set them per cell.

        column_headers = summary_df_hierarchical.columns.tolist()
        # Header Row
        for j, header in enumerate(column_headers):
            cell = table.cell(0, j)
            paragraph = cell.paragraphs[0]
            run = paragraph.add_run(header)
            run.font.bold = True
            run.font.name = font_name
            run.font.size = font_size
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            # Set bottom border for header cells, remove others
            set_cell_borders(cell, 
                             bottom={"sz": 6, "val": "single", "color": "000000"}, # 6 for 0.75pt, "000000" for black
                             top={"val": "nil"}, left={"val": "nil"}, right={"val": "nil"})

        # Data Rows
        num_data_rows = summary_df_hierarchical.shape[0]
        for i, row_data in summary_df_hierarchical.iterrows():
            is_parent_category_row = not str(row_data[column_headers[0]]).startswith("  ")
            
            for j, col_name in enumerate(column_headers):
                cell_text = str(row_data[col_name])
                cell = table.cell(i + 1, j)
                paragraph = cell.paragraphs[0]
                run = paragraph.add_run(cell_text)
                run.font.name = font_name
                run.font.size = font_size
                paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

                if j == 0: # First column for category names
                    if is_parent_category_row:
                        run.font.bold = True
                        paragraph.paragraph_format.left_indent = parent_indent
                    else: # Subcategory
                        # The text already has "  - ". The indent adds to it.
                        paragraph.paragraph_format.left_indent = sub_cat_indent
                
                # Border logic for data rows
                border_args = {"left": {"val": "nil"}, "right": {"val": "nil"}, "top": {"val": "nil"}, "bottom": {"val": "nil"}}
                if (i + 1) == num_data_rows: # If it's the last data row
                    border_args["bottom"] = {"sz": 6, "val": "single", "color": "000000"}
                
                set_cell_borders(cell, **border_args)
        
        # Optional: Set preferred column widths (can fine-tune this)
        # widths = (Inches(3.5), Inches(2.5), Inches(1.5))
        # for idx, width in enumerate(widths):
        #     if idx < len(table.columns):
        #         table.columns[idx].width = width


        file_path = "outputs/Summary of Protocol Deviations.docx"
        document.save(file_path)
        print(f"\nWord document '{file_path}' created with custom borders and formatting.")

    except Exception as e:
        print(f"An error occurred while creating the Word document: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\nSummary DataFrame ('summary_df_hierarchical') is empty or not defined. Cannot create Word document.")
